<a href="https://colab.research.google.com/github/japeshmethuku17/Sarcasm-Detection/blob/master/sarcasm_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sarcasm detection using Natural Language Processing**

Run the next cell to ensure TensorFlow v2.x exists in your Google Colab notebook.

In [0]:
try:
  %tensorflow_version 2.x
  print("Version available")
except Exception:
  pass
  print("Version not available")

Version available


In [0]:
import json
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import model_from_json

In [0]:
vocab_size = 10000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000


This method is used to retreive the dataset. The json file is then loaded into the Colab notebook.

In [0]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json \
    -O /tmp/sarcasm.json


--2020-06-11 17:53:52--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.204.128, 2404:6800:4008:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.204.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘/tmp/sarcasm.json’

/tmp/sarcasm.json   100%[===================>]   5.38M  --.-KB/s    in 0.04s   

2020-06-11 17:53:52 (146 MB/s) - ‘/tmp/sarcasm.json’ saved [5643545/5643545]



Processing the dataset, the columns 'headline' and 'is_sarcastic' is important to us. There is another column named 'article_link' and it is not significant at this point. Our goal is to predict the sarcasm using the headlines.

In [0]:
with open("/tmp/sarcasm.json", 'r') as f:
    datastore = json.load(f)

sentences = []
labels = []

for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])

We are splitting the dataset into training and testing sets using the method shown below.

In [0]:
training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

The Machine Learning approach requires numerical feature vectors to perform a task. The idea of pre-processing the text data is to make the data ideal for a Machine Learning algorithm to work. The preprocessing can include converting the entire text data into uppercase or lowercase, this creates a uniformity and the entire text data is treated in the same fashion.

**Tokenization:** This process is used to convert the normal text strings into a list of tokens. 



**Removal of noise:** Something that can't be regarded as a number or a letter is removed before feeding the data into the NLP model.

**Removal of stop words:** Removal of common words that add very little meaning to the data.

In [0]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [0]:
import numpy as np
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

Create a neural network model.

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

Display the structure of model.

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 16)           160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 24)                408       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25        
Total params: 160,433
Trainable params: 160,433
Non-trainable params: 0
_________________________________________________________________


Train the model for 30 epochs

In [0]:
num_epochs = 30
history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=2)

Epoch 1/30
625/625 - 4s - loss: 0.6708 - accuracy: 0.5807 - val_loss: 0.6010 - val_accuracy: 0.6350
Epoch 2/30
625/625 - 3s - loss: 0.4354 - accuracy: 0.8302 - val_loss: 0.3893 - val_accuracy: 0.8326
Epoch 3/30
625/625 - 3s - loss: 0.3106 - accuracy: 0.8755 - val_loss: 0.3566 - val_accuracy: 0.8436
Epoch 4/30
625/625 - 3s - loss: 0.2578 - accuracy: 0.9001 - val_loss: 0.3457 - val_accuracy: 0.8542
Epoch 5/30
625/625 - 3s - loss: 0.2236 - accuracy: 0.9137 - val_loss: 0.3433 - val_accuracy: 0.8533
Epoch 6/30
625/625 - 3s - loss: 0.1952 - accuracy: 0.9251 - val_loss: 0.3562 - val_accuracy: 0.8489
Epoch 7/30
625/625 - 3s - loss: 0.1730 - accuracy: 0.9339 - val_loss: 0.3691 - val_accuracy: 0.8481
Epoch 8/30
625/625 - 3s - loss: 0.1554 - accuracy: 0.9420 - val_loss: 0.3794 - val_accuracy: 0.8521
Epoch 9/30
625/625 - 3s - loss: 0.1399 - accuracy: 0.9493 - val_loss: 0.3967 - val_accuracy: 0.8521
Epoch 10/30
625/625 - 3s - loss: 0.1283 - accuracy: 0.9557 - val_loss: 0.4175 - val_accuracy: 0.8502

In [0]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_sentence(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

decode_sentence(training_padded[0])

In [0]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)


(10000, 16)


In [0]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

Making predictions on unseen data.

In [0]:
sentence = ["Mirrors can’t talk, lucky for you they can’t laugh either"]
sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
result=model.predict(padded)
print(result)

[[0.9999863]]


Saving the model

In [0]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
  json_file.write(model_json)
model.save("saved_model.h5")

Loading the model

In [0]:
# opening and store file in a variable

json_file = open('model.json','r')
loaded_model_json = json_file.read()
json_file.close()

# use Keras model_from_json to make a loaded model

loaded_model = model_from_json(loaded_model_json)

# load weights into new model

loaded_model.load_weights("saved_model.h5")
print("Loaded Model from disk")

Loaded Model from disk
